# IMPORT

In [2]:
from pybit.unified_trading import HTTP
from datetime import datetime
import math
import talib
import numpy as np
import pymysql
import cryptography

# API KEY

In [3]:
# api key
with open("C:\\Users\\lee\\Desktop\\bybit\\api.txt", "r") as file:
    api_list = []
    # 파일의 각 줄에 대해 반복
    for line in file:
        # 각 줄 출력
        api_list.append(line.strip())  # strip()은 줄 바꿈 문자(\n) 등을 제거하여 출력합니다.

# sql pw
with open("C:\\Users\\lee\\Desktop\\bybit\\pw.txt", "r") as file:
        db_password = file.readline()

key = api_list[0]
secret = api_list[1]

# 트레이딩 정보 가져오기 / 교차, 격리, 레버리지 배율 수정

In [11]:
def trading_info(key, secret, coin_symbol, leverage):

    # api 적용 세션
    session = HTTP(
        # testnet=True,
        api_key=key,
        api_secret=secret,
    )

    # 트레이딩 정보 가져오기
    trade_mode = session.get_positions(
        category="linear",
        symbol=coin_symbol,
    )

    cross_isolated = trade_mode['result']['list'][0]['tradeMode']

    if cross_isolated == 1:
        return print('모드 : 격리')
    else:
        # 교차, 격리 / 레버리지 배율 수정
        session.switch_margin_mode(
        category="linear",
        symbol=coin_symbol,
        tradeMode=1, # 0: cross margin. 1: isolated margin
        buyLeverage=leverage,
        sellLeverage=leverage,
        )
        return print('모드 : 격리로 변경')



# 기술적 지표

In [11]:
def rsi_indicator(closePrice_reverse, period):
    # RSI
    price_data = np.array(closePrice_reverse) # 파라미터

    rsi_period = period # 파라미터
    rsi_values = talib.RSI(price_data, timeperiod=rsi_period)

    return rsi_values[-1]

In [12]:
def bb_indicator(closePrice, period, up, down):
    # Bollinger Bands
    price_data = np.array(closePrice) # 파라미터

    upperband, middleband, lowerband = talib.BBANDS(price_data, period, up, down)

    return upperband[-1], middleband[-1], lowerband[-1]

# 캔들 마크 가격

In [13]:
def candle_mark_price(coin_symbol):

    # 현재 시각 구하기
    now = datetime.now()
    now_timestamp = math.floor(now.timestamp()*1000)

    # get_kline / get_mark_price_kline
    session = HTTP(testnet=True)
    price = session.get_mark_price_kline(
        category="linear",
        symbol=coin_symbol,
        interval=1,
        start=now_timestamp-60000000,
        end=now_timestamp,
        limit=1000,
    )

    # price['result']['list'][:1001]

    a_hundred_prices = price['result']['list']
    highPrice = []
    lowPrice = []
    closePrice = []
    for i in range(len(a_hundred_prices)):
        highPrice.append(float(a_hundred_prices[i][2]))
        lowPrice.append(float(a_hundred_prices[i][3]))
        closePrice.append(float(a_hundred_prices[i][4]))

    # 가장 최신 가격을 앞에서 뒤로 변경하기 위한 코드
    highPrice_reverse = highPrice[::-1]
    lowPrice_reverse = lowPrice[::-1]
    closePrice_reverse = closePrice[::-1]

    return highPrice_reverse, lowPrice_reverse, closePrice_reverse


# QTY, TP/SL

In [14]:
def find_first_digit_after_decimal(str_number):
    # 소수점 이하의 숫자 추출
    decimal_part = str(str_number).split('.')[1]
    
    # 1의 자리 숫자 찾기
    for i in range(len(decimal_part)):
        if decimal_part[i] == '1':
            return i + 1  # 1의 자리이므로 인덱스 + 1 반환
    
    return None  # 1의 자리 숫자가 없는 경우

In [15]:
def qty_calculator(key, secret, coin_symbol, now_close_price, position_side):
    session = HTTP(
        # testnet=True,
        api_key=key,
        api_secret=secret,
    )
    # 사용 가능한 usdt 값
    for_usdt = session.get_wallet_balance(
        accountType="CONTRACT",
        coin='USDT',
    )

    available_usdt = for_usdt['result']['list'][0]['coin'][0]['availableToWithdraw']
    available_usdt = float(available_usdt)

    # 최소 주문 금액에 따른 소수점 버리기 및 정수화
    session = HTTP(testnet=True)
    instruments_info = session.get_instruments_info(
        category="linear",
        symbol=coin_symbol,
    )

    minOrderQty = instruments_info['result']['list'][0]['lotSizeFilter']['minOrderQty']
    minOrderQty = float(minOrderQty)

    if minOrderQty == 0.001:
        result_qty = available_usdt / now_close_price
        result_qty = "{:.3f}".format(result_qty)

    elif minOrderQty == 0.01:
        result_qty = available_usdt / now_close_price
        result_qty = "{:.2f}".format(result_qty)

    elif minOrderQty == 0.1:
        result_qty = available_usdt / now_close_price
        result_qty = "{:.1f}".format(result_qty)

    else:
        result_qty = available_usdt / now_close_price
        result_qty = int(result_qty)
        result_qty = str(result_qty)

    # 최소 주문 금액
    minPrice = instruments_info['result']['list'][0]['priceFilter']['minPrice']

    # 1의 자리수 
    one_seat = find_first_digit_after_decimal(minPrice)

    # TP / SL 변수 값
    if position_side == "Buy":
        tp_price = now_close_price + (now_close_price * 0.01)
        tp_price = str(tp_price)
        tp_price = tp_price[:one_seat+2]
        tp_price = float(tp_price)

        sl_price = now_close_price - (now_close_price * 0.005)
        sl_price = str(sl_price)
        sl_price = sl_price[:one_seat+2]
        sl_price = float(sl_price)

    else:
        tp_price = now_close_price - (now_close_price * 0.01)
        tp_price = str(tp_price)
        tp_price = tp_price[:one_seat+2]
        tp_price = float(tp_price)

        sl_price = now_close_price + (now_close_price * 0.005)
        sl_price = str(sl_price)
        sl_price = sl_price[:one_seat+2]
        sl_price = float(sl_price)
        

    return available_usdt, result_qty, tp_price, sl_price

# 롱 / 숏 판별

In [16]:
def positon_info(key, secret, coin_symbol):
    session = HTTP(
        # testnet=True,
        api_key=key,
        api_secret=secret,
    )
    side_check = session.get_positions(
        category="linear",
        symbol=coin_symbol,
    )

    return side_check['result']['list'][0]['side']

# 주문

In [17]:
def order_execution(key, secret, coin_symbol, order_side, result_qty, tp_price, sl_price):

    session = HTTP(
        # testnet=True,
        api_key=key,
        api_secret=secret,
    )

    order_session = session.place_order(
        category="linear",
        symbol=coin_symbol,
        side=order_side, 
        orderType="Market",
        qty=result_qty, 
        timeInForce="IOC", 
        isLeverage=1,
        orderFilter="Order",
        
        # TP / SL
        takeProfit = tp_price,
        stopLoss = sl_price,
    )

    oder_time = datetime.utcfromtimestamp(order_session['time']/1000)

    return oder_time

# MySQL

In [18]:
def order_db_record(db_password, coin_symbol, order_time, now_close_price, available_usdt, result_qty, tp_price, sl_price, rsi_14_number, bb_number):

    conn = pymysql.connect(host='127.0.0.1', user='root', password=db_password, db='bybit', charset='utf8') 

    # 커서 생성
    cursor = conn.cursor()

    # 테이블 생성 쿼리
    create_table_query = """
    CREATE TABLE IF NOT EXISTS orders (
        id INT AUTO_INCREMENT PRIMARY KEY,
        coin_symbol VARCHAR(20),
        order_time VARCHAR(40),
        now_close_price FLOAT,
        available_usdt FLOAT,
        result_qty FLOAT,
        tp_price FLOAT,
        sl_price FLOAT,
        rsi_14_number FLOAT,
        bb_number FLOAT
    )
    """
    cursor.execute(create_table_query)

    # 주문 정보 삽입 쿼리
    insert_query = """
    INSERT INTO orders (coin_symbol, order_time, now_close_price, available_usdt, result_qty, tp_price, sl_price, rsi_14_number, bb_number)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # 주문 정보
    order_data = (coin_symbol, order_time, now_close_price, available_usdt, result_qty, tp_price, sl_price, rsi_14_number, bb_number)

    # 쿼리 실행
    cursor.execute(insert_query, order_data)

    # 변경사항 저장
    conn.commit()

    # 연결 종료
    cursor.close()
    conn.close()




# TEST - 트레이딩

In [19]:
coin_symbol = 'MNTUSDT'

# leverage 조정하기
trading_info(key, secret, coin_symbol, 1)

# while True:
highPrice_reverse, lowPrice_reverse, closePrice_reverse = candle_mark_price(coin_symbol)

rsi_14_number = rsi_indicator(closePrice_reverse, 14)
upperband, middleband, lowerband = bb_indicator(closePrice_reverse, 20, 2, 2)

now_close_price = closePrice_reverse[-1] # 값 확인하기

# 롱 포지션
if (50.1 <= rsi_14_number <= 55) and (now_close_price > middleband):
    position_side = "Buy"

elif (45 <= rsi_14_number <= 49.9) and (now_close_price > middleband):
    position_side = "Sell"


모드 : 격리


In [51]:
from pybit.unified_trading import HTTP

session = HTTP(
    # testnet=True,
    api_key=key,
    api_secret=secret,
)

order_session = session.place_order(
    category="linear",
    symbol='MNTUSDT',
    side='Buy', 
    orderType="Limit",
    qty='1', 
    timeInForce="GTC", 
    isLeverage=1,
    orderFilter="Order",
    orderLinkId = "test15",
    price = '0.5',
    
    # TP / SL
    takeProfit = 0.55,
    stopLoss = 0.48,
)

In [52]:
order_session

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'orderId': '5a05fa17-5d54-4630-b71d-e9f1b175aa5f',
  'orderLinkId': 'test15'},
 'retExtInfo': {},
 'time': 1700934444301}

In [55]:
from pybit.unified_trading import HTTP
session = HTTP(
    # testnet=True,
    api_key=key,
    api_secret=secret,
)
print(session.amend_order(
    category="linear",
    symbol="MNTUSDT",
    # orderLinkId="201c7349-4a0c-4725-afe5-9ae05f557e53",
    orderLinkId = 'test15',
    price = '0.51423',
    takeProfit = 0.51498,
    stopLoss = 0.51349,
))

InvalidRequestError: Order does not exist (ErrCode: 110001) (ErrTime: 17:49:41).
Request → POST https://api.bybit.com/v5/order/amend: {"category": "linear", "symbol": "MNTUSDT", "orderLinkId": "test15", "price": "0.51423", "takeProfit": "0.51498", "stopLoss": "0.51349"}.